In [ ]:
from torch.nn.functional import cosine_similarity
import torch as th
import numpy as np
import pandas as pd
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

th.set_grad_enabled(False)

In [ ]:
device = "cuda"

In [ ]:
import numpy as np
if not hasattr(np, "bool"):
    np.bool = bool 

In [ ]:
import json

In [ ]:
def load_dictionary_model(
    model_name: str | Path, is_sae: bool | None = None,
):
    """Load a dictionary model from a local path or HuggingFace Hub.

    Args:
        model_name: Name or path of the model to load

    Returns:
        The loaded dictionary model
    """

    # Local model
    model_path = Path(model_name)
    if not model_path.exists():
        raise ValueError(f"Local model {model_name} does not exist")

    # Load the config
    with open(model_path.parent / "config.json", "r") as f:
        config = json.load(f)["trainer"]

    # Determine model class based on config
    if "dict_class" in config and config["dict_class"] in [
        "BatchTopKSAE",
        "CrossCoder",
        "BatchTopKCrossCoder",
    ]:
        return eval(f"{config['dict_class']}.from_pretrained(model_path)")
    else:
        raise ValueError(f"Unknown model type: {config['dict_class']}")


In [ ]:
from dictionary_learning.dictionary import BatchTopKCrossCoder, CrossCoder

In [ ]:
coder = load_dictionary_model("crosscoder_checkpoints/Qwen3-1.7B-L20-k100-lr1e-04-ep2-run_1-Crosscoder/checkpoint_90000.pt")

In [ ]:
base_layer, instruct_layer = 0,1 

In [ ]:
df = pd.read_csv("latent_df.csv")

In [ ]:
deads = (df.frequencies_true.values == 0) & (df.frequencies_false.values == 0)

In [ ]:
num_layers, activation_dim, dict_size = coder.encoder.weight.shape
alive_mask = ~deads

In [ ]:
# Create masks for each category
finetune_only_mask = norm_diffs[alive_mask] <= 0.2
base_only_mask = norm_diffs[alive_mask] >= 0.8
shared_mask = (0.4 <= norm_diffs[alive_mask]) & (norm_diffs[alive_mask] <= 0.6)
other_mask = ~(finetune_only_mask | base_only_mask | shared_mask)

# Create figure
fig = go.Figure()

# Add histogram traces for each category
fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][finetune_only_mask],
        name="Finetune only",
        marker_color="green",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][base_only_mask],
        name="Base only",
        marker_color="orange",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][shared_mask],
        name="Shared",
        marker_color="blue",
        nbinsx=50,
    )
)

fig.add_trace(
    go.Histogram(
        x=norm_diffs[alive_mask][other_mask],
        name="Other",
        marker_color="grey",
        nbinsx=50,
    )
)

fig.update_layout(
    annotations=[
        dict(
            x=0,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Finetune only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="green"),
        ),
        dict(
            x=0.5,
            y=0,
            xref="x",
            yref="paper",
            text="    <b>(shared)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="blue"),
        ),
        dict(
            x=1,
            y=0,
            xref="x",
            yref="paper",
            text="  <b>(Base only)</b>",
            showarrow=False,
            yanchor="top",
            xanchor="left",
            font=dict(color="orange"),
        ),
    ],
    shapes=[
        dict(  # Finetune threshold
            type="line",
            x0=0.2,
            y0=0,
            x1=0.2,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="green", width=2),
        ),
        dict(  # Base threshold
            type="line",
            x0=0.8,
            y0=0,
            x1=0.8,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="orange", width=2),
        ),
        dict(  # Shared lower
            type="line",
            x0=0.4,
            y0=0,
            x1=0.4,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="blue", width=2),
        ),
        dict(  # Shared upper
            type="line",
            x0=0.6,
            y0=0,
            x1=0.6,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="blue", width=2),
        ),
    ],
    showlegend=False,
)

fig.update_xaxes(tickvals=[0, 0.5, 1])
fig.update_xaxes(title="Sorted Features (Highest to Lowest Difference)")
fig.update_yaxes(title="Relative Norm Difference", range=[0, 50])
fig.update_layout(barmode="stack")
fig.show()

In [ ]:
df['dec_norm_diffs'] = norm_diffs

In [ ]:
conditions = [
    df["dec_norm_diffs"] <= 0.2,
    (df["dec_norm_diffs"] >= 0.4) & (df["dec_norm_diffs"] <= 0.6),
    df["dec_norm_diffs"] >= 0.8,
]
choices = ["Finetune_only", "Shared", "Base_only"]

df["latent_tag"] = np.select(conditions, choices, default="Other")

In [ ]:
df.to_csv('latent_df.csv',index=False)